In [22]:
! pip install cohere langchain qdrant-client tfds-nightly python-dotenv pymongo pandas numpy modal modal-client

In [23]:

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate, LLMChain
import textwrap as tr
import random
import dotenv
import os
import pymongo
import pandas as pd
from tqdm import tqdm

dotenv.load_dotenv(".env")

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
MONGO_URI = os.getenv("MONGO_URI")

mongodb_client = pymongo.MongoClient(MONGO_URI, serverSelectionTimeoutMS=3_600_000)

# Create dataset and collection
db = mongodb_client.cupidgpt
collection = db.profiles

### Load data into MongoDB

In [2]:
df = pd.read_csv("data/profile_cleaned.csv")
df.head()

/var/folders/45/zrvxxdqn4sv6sz8mcdhkbv840000gn/T/ipykernel_94673/2248988785.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/profile_cleaned.csv")


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,last_online,location,offspring,pets,religion,sign,smokes,speaks,About_Me,User_ID
0,22,single,male,straight,a little extra,strictly anything,drinks socially,never does drugs,working on college/university,"asian, white",...,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,smokes sometimes,english,currently working as an international agent fo...,1
1,35,single,male,straight,average,mostly other,drinks often,sometimes does drugs,working on space camp,white,...,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (...",dedicating everyday to being an unbelievable b...,2
2,38,available,male,straight,thin,anything,drinks socially,Not Available,graduated from masters program,Not Available,...,2012-06-27-09-10,"san francisco, california",Not Available,has cats,Not Available,pisces but it doesnot matter,no,"english, french, c++","i make nerdy software for musicians, artists, ...",3
3,23,single,male,straight,thin,vegetarian,drinks socially,Not Available,working on college/university,white,...,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes cats,Not Available,pisces,no,"english, german (poorly)",reading things written by old dead people. rea...,4
4,29,single,male,straight,athletic,Not Available,drinks socially,never does drugs,graduated from college/university,"asian, black, other",...,2012-06-27-21-26,"san francisco, california",Not Available,likes dogs and likes cats,Not Available,aquarius,no,english,work work work work + play. work work work wor...,5


In [13]:
# Save dataframe in MongoDB
mongodb_client = pymongo.MongoClient(MONGO_URI, serverSelectionTimeoutMS=3_600_000)

# Create dataset and collection
db = mongodb_client.cupidgpt
collection = db.profiles

# Insert dataframe into collection
# Insert in batches of 10
# for i in range(0, len(df), 10):
#     if i+10 < len(df):
#         df_batch = df[i:i+10]
#     else:
#         df_batch = df[i:]
#     collection.insert_many(df_batch.to_dict("records"))
collection.insert_many(df.to_dict("records"))

# Delete all documents in collection
# collection.delete_many({})

# Get number of items in collection
collection.count_documents({})



In [18]:
# Get user_id 35 from collection
print(collection.find_one({"User_ID": 35}))


{'_id': ObjectId('64ea6a799c9fa34cea272636'), 'age': 27, 'status': 'single', 'sex': 'male', 'orientation': 'straight', 'body_type': 'thin', 'diet': 'strictly anything', 'drinks': 'drinks socially', 'drugs': 'never does drugs', 'education': 'graduated from college/university', 'ethnicity': 'white', 'height': 69.0, 'income': -1, 'job': 'entertainment / media', 'last_online': '2012-06-24-22-10', 'location': 'san francisco, california', 'offspring': 'Not Available', 'pets': 'likes dogs and likes cats', 'religion': 'atheism and somewhat serious about it', 'sign': 'sagittarius but it doesnot matter', 'smokes': 'smokes when drinking', 'speaks': 'english', 'About_Me': "i'm a cinematic artist for a video game studio. i love being in a creative environment surrounded by so many talented coworkers who challenge me to always be improving my work. this job has cultivated a strong passion for storytelling which has given me a bigger appetite for movies, books, and tv. i love discussing these and wha

### Cohere GenAI LLM - Test

In [31]:
template = """
Create a descriptive "About me" for this user of a dating app. 
Write it like the person would have written about themselves to describe themself for the app. 
Use all fields from the following JSON data.
User Data: {user_data}
"""

prompt = PromptTemplate(template=template, input_variables=["user_data"])

llm = Cohere(cohere_api_key=COHERE_API_KEY, model="command", max_tokens=1000, temperature=0.5)
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Get user_id 35 from collection
user_data = collection.find_one({"User_ID": 35})
llm_chain.run(user_data=user_data)




' I am a 27-year-old male who is single and straight. I have a thin body type and I am 5\'9". I have a college degree and I am an entertainment/media professional. I live in San Francisco, California and I am a Sagittarius. I love to discuss movies, books, and TV shows. I am a passionate storyteller and I love to be challenged to improve my work. I am a big fan of dogs and cats. I am an atheist and I am serious about it. I am a social drinker and I do not do drugs. I love to laugh and I am a positive person. I am a good friend and I am a great listener. I am a romantic at heart and I am looking for someone to share my life with. I am a cinematic artist for a video game studio. I love being in a creative environment surrounded by so many talented coworkers who challenge me to always be improving my work. This job has cultivated a strong passion for storytelling which has given me a bigger appetite for movies, books, and tv. I love discussing these and what makes them good or bad, why th

### Cohere GenAI LLM - Scale

In [ ]:

# see all User_ID in collections
about_me_generated = []
# use tqdm

for user in tqdm(collection.find()):
    # print(user["User_ID"])


    user_data = collection.find_one({"User_ID": user["User_ID"]})
    about_me_generated.append(llm_chain.run(user_data=user_data))
    # if user is last
    # if user["User_ID"] == collection.count_documents({}):
    #     print(llm_chain.output)

### Modal Encoding

In [12]:
import requests
from modal import Stub, Image, method

stub = Stub()

def download_models():
    # Caches the model inside the Modal image, so subsequent cold starts are faster.
    from sentence_transformers import SentenceTransformer
    SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

image = Image.debian_slim().pip_install("sentence-transformers").run_function(download_models)

@stub.cls(image=image)
class ModalEmbedding:
    def __enter__(self):
      from sentence_transformers import SentenceTransformer
      self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    @method()
    def generate(self, text: str):
      return self.model.encode(text).tolist()

In [11]:
import modal

stub = modal.Stub("example-get-started")


@stub.function()
def square(x):
    print("This code is running on a remote worker!")
    return x**2


@stub.local_entrypoint()
def main():
    print("the square is", square.remote(42))


In [12]:
stub.run("main")